# Perform prediction

## Import libraries

In [1]:
import os

import logging
import sys

log = logging.getLogger("Pipeline")
log.setLevel(logging.INFO)
format = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")

ch = logging.StreamHandler(sys.stdout)
ch.setFormatter(format)
log.addHandler(ch)

import warnings
warnings.simplefilter(action='ignore', category=Warning)

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import predict_mortality as pm

## Define data directory

A top level directory to store all the data for this experiment

In [2]:
dataDirName = '/home/yram0006/phd/chapter_2/workspace/mortality_data/imbalance_experiment/'

## Read data matrix

In [3]:
import pandas as pd

dataMatrix = pd.read_csv(dataDirName + 'data_matrix/data_matrix_standardised.csv')
pd.set_option('display.max_columns', None)
dataMatrix

,person_id,age,gender,ethnicity_WHITE,ethnicity_BLACK,ethnicity_UNKNOWN,ethnicity_OTHER,ethnicity_HISPANIC,ethnicity_ASIAN,ethnicity_UNABLE_TO_OBTAIN,ethnicity_AMERICAN_INDIAN,anchor_time,death_datetime,heartrate_min,sysbp_min,diabp_min,meanbp_min,resprate_min,tempc_min,spo2_min,gcseye_min,gcsverbal_min,gcsmotor_min,heartrate_max,sysbp_max,diabp_max,meanbp_max,resprate_max,tempc_max,spo2_max,gcseye_max,gcsverbal_max,gcsmotor_max,heartrate_avg,sysbp_avg,diabp_avg,meanbp_avg,resprate_avg,tempc_avg,spo2_avg,gcseye_avg,gcsverbal_avg,gcsmotor_avg,heartrate_stddev,sysbp_stddev,diabp_stddev,meanbp_stddev,resprate_stddev,tempc_stddev,spo2_stddev,gcseye_stddev,gcsverbal_stddev,gcsmotor_stddev,heartrate_first,sysbp_first,diabp_first,meanbp_first,resprate_first,tempc_first,spo2_first,gcseye_first,gcsverbal_first,gcsmotor_first,heartrate_last,sysbp_last,diabp_last,meanbp_last,resprate_last,tempc_last,spo2_last,gcseye_last,gcsverbal_last,gcsmotor_last,chloride_serum_min,creatinine_min,sodium_serum_min,hemoglobin_min,platelet_count_min,urea_nitrogen_min,glucose_serum_min,bicarbonate_min,potassium_serum_min,anion_gap_min,leukocytes_blood_manual_min,hematocrit_min,chloride_serum_max,creatinine_max,sodium_serum_max,hemoglobin_max,platelet_count_max,urea_nitrogen_max,glucose_serum_max,bicarbonate_max,potassium_serum_max,anion_gap_max,leukocytes_blood_manual_max,hematocrit_max,chloride_serum_avg,creatinine_avg,sodium_serum_avg,hemoglobin_avg,platelet_count_avg,urea_nitrogen_avg,glucose_serum_avg,bicarbonate_avg,potassium_serum_avg,anion_gap_avg,leukocytes_blood_manual_avg,hematocrit_avg,chloride_serum_stddev,creatinine_stddev,sodium_serum_stddev,hemoglobin_stddev,glucose_serum_stddev,bicarbonate_stddev,potassium_serum_stddev,chloride_serum_first,creatinine_first,sodium_serum_first,hemoglobin_first,platelet_count_first,urea_nitrogen_first,glucose_serum_first,bicarbonate_first,potassium_serum_first,anion_gap_first,leukocytes_blood_manual_first,hematocrit_first,chloride_serum_last,creatinine_last,sodium_serum_last,hemoglobin_last,platelet_count_last,urea_nitrogen_last,glucose_serum_last,bicarbonate_last,potassium_serum_last,anion_gap_last,leukocytes_blood_manual_last,hematocrit_last
0,-2144679073,82.0,0,0,0,0,1,0,0,0,0,2190-01-30 19:22:00,2194-04-23 19:27:00,-0.272794,1.217771,0.923258,0.861968,0.878048,0.189353,0.317249,0.412549,1.239978,0.878154,-0.030099,-0.001925,0.069606,0.021527,-0.309282,-0.058429,-0.021341,0.370578,0.669108,0.341538,-0.671766,0.838925,0.734657,0.755136,0.000510,-0.003616,-0.100585,0.595497,1.041441,0.658405,-0.047481,-0.000154,0.026722,-0.025099,-0.700168,-0.113932,-0.042762,-0.054623,-0.696918,-0.769890,-1.241070,-0.100946,-0.093761,-0.277365,-0.176019,-0.090217,-0.015482,0.729111,0.910102,0.608975,-0.521863,1.715295,0.616779,0.846636,-0.029003,-0.048489,0.009782,0.659401,0.896608,0.536315,1.093242,-0.663478,0.163419,0.285937,0.076969,-0.734973,-0.012647,0.669296,-0.124032,-0.915664,-0.348052,0.099873,0.214809,-0.011713,-0.309154,-0.534404,0.297850,-0.994593,-0.024332,-0.000648,-1.025656,-0.767333,-0.018388,-0.395681,-0.007656,-0.014171,-0.015245,-0.012701,0.347919,-0.948893,-0.020354,0.019154,-0.013272,-1.114836,-0.162093,-0.233373,-0.012429,-0.011540,-0.590509,-0.011974,-0.030961,-0.601701,-1.056430,0.743687,-0.694348,-0.191987,-0.406075,0.116251,-0.789275,-0.022292,0.207578,-0.788282,-1.346000,-0.054095,-0.572535,0.656699,-0.695650,-0.087780,0.157334,-0.178773,-0.863655,-0.015096,0.214490,-0.408989,-0.979015,-0.516496,0.100735
1,-2142084288,84.0,1,1,0,0,0,0,0,0,0,2161-07-10 08:07:00,NaN,-0.073643,-0.427373,0.222451,-1.935422,1.165394,0.217516,-0.256040,1.199315,1.239978,0.878154,-0.004134,-0.014919,-0.026310,-0.053989,-0.057963,-0.040074,-0.021341,0.370578,0.669108,0.341538,-0.134094,-0.066312,0.750897,0.395390,0.617088,0.086249,-0.167731,0.860623,1.041441,0.658405,-0.001199,0.007246,0.014587,0.037638,-0.347862,-0.101546,-0.018473,-1.069636,-0.696918,-0.769890,-0.299599,0.244989,0.060789,0.378146,0.701921,0.305525,-0.05

## Perform predictions

### Calculate target variable

In [4]:
dataMatrix.anchor_time = dataMatrix.anchor_time.apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S'))
dataMatrix.death_datetime = dataMatrix.death_datetime.apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S'))
dataMatrix['target'] = (dataMatrix['death_datetime'] < (dataMatrix['anchor_time'] + pd.Timedelta(days=7)))
dataMatrix.target.fillna(value=False, inplace=True)

### Check class counts

In [5]:
dataMatrix.target.value_counts()

target
False    2524
True      447
Name: count, dtype: int64

### Separate positive and negative classes

In [6]:
dataMatrixPositive = dataMatrix[dataMatrix.target == True]
dataMatrixNegative = dataMatrix[dataMatrix.target == False]

### Simulate different data imbalances and predict

In [7]:
totalSamples = 1200
for proportion in [65, 70, 75, 80, 85, 90, 95]:
    negativeSize = int(totalSamples * proportion / 100)
    positiveSize = int(totalSamples - (totalSamples * proportion / 100))
    sampledDataMatrix = pd.concat([dataMatrixPositive.sample(n=positiveSize), dataMatrixNegative.sample(n=negativeSize)]).sample(frac=1).reset_index(drop=True)
    sampledDataMatrix.to_csv(dataDirName + 'data_matrix.csv', index=False)
    pm.runPredictionsForAllTargets(
        label=str(proportion) + '_' + str(100 - proportion) + '_samples',
        dirPath = dataDirName,
        vitalsBefore = 0,
        vitalsAfter = 72,
        labsBefore = 0,
        labsAfter = 72,
        targetList = [7],
    )


2023-09-22 21:37:51,776 - Pipeline - INFO - dirName: 65_35_samples_icd_vb_0_va_72_lb_0_la_72
2023-09-22 21:37:51,777 - Pipeline - INFO - Running Predictions for vb_0_va_72_lb_0_la_72, targetStart : 0, targetEnd : 7
2023-09-22 21:37:51,778 - Pipeline - INFO - Reading data
2023-09-22 21:37:51,994 - Pipeline - INFO - Formatting data
2023-09-22 21:37:52,000 - Pipeline - INFO - Performing SFS
2023-09-22 21:39:21,566 - Pipeline - INFO - Building XGBoost model with all the features
2023-09-22 21:39:21,568 - Pipeline - INFO - Performing Hyperparameter optimisation
2023-09-22 21:39:21,592 - Pipeline - INFO - Building the model
2023-09-22 21:39:21,593 - Pipeline - INFO - Hyperparameter optimisation for: {'max_depth': range(1, 10), 'scale_pos_weight': [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]}
2023-09-22 21:40:21,996 - Pipeline - INFO - Hyperparameter optimisation for: {'n_estimators': range(50, 250, 10)}
2023-09-22 21:40:48,713 - Pipeline - INFO - Hyperparameter optimisation for: {'min_child_weight

R[write to console]: Error in ROCR::prediction(predictor, response) : 
  Number of classes is not equal to 2.
ROCR currently supports only evaluation of binary classification tasks.

R[write to console]: In addition: 
R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 

R[write to console]: Error in ROCR::prediction(predictor, response) : 
  Number of classes is not equal to 2.
ROCR currently supports only evaluation of binary classification tasks.

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  libraries ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ contain no packages

R[write to console]: Error in ROCR::prediction(predictor, response) : 
  Number of classes is not equal to 2.
ROCR currently supports only evaluation of binary classification 

2023-09-22 22:11:34,658 - Pipeline - INFO - Building Ensemble LR model with all the features
2023-09-22 22:11:34,659 - Pipeline - INFO - Performing Hyperparameter optimisation
2023-09-22 22:11:35,294 - Pipeline - INFO - Building the model
2023-09-22 22:11:35,296 - Pipeline - INFO - Performing cross-validation
2023-09-22 22:11:36,134 - Pipeline - INFO - Saving the CV results for all the models
2023-09-22 22:11:36,137 - Pipeline - INFO - Completed !!!
2023-09-22 22:11:36,349 - Pipeline - INFO - dirName: 80_20_samples_icd_vb_0_va_72_lb_0_la_72
2023-09-22 22:11:36,350 - Pipeline - INFO - Running Predictions for vb_0_va_72_lb_0_la_72, targetStart : 0, targetEnd : 7
2023-09-22 22:11:36,351 - Pipeline - INFO - Reading data
2023-09-22 22:11:36,615 - Pipeline - INFO - Formatting data
2023-09-22 22:11:36,623 - Pipeline - INFO - Performing SFS
2023-09-22 22:13:11,328 - Pipeline - INFO - Building XGBoost model with all the features
2023-09-22 22:13:11,329 - Pipeline - INFO - Performing Hyperparame

R[write to console]: Error in ROCR::prediction(predictor, response) : 
  Number of classes is not equal to 2.
ROCR currently supports only evaluation of binary classification tasks.

R[write to console]: In addition: 
R[write to console]: There were 11 warnings (use warnings() to see them)
R[write to console]: 

R[write to console]: Error in ROCR::prediction(predictor, response) : 
  Number of classes is not equal to 2.
ROCR currently supports only evaluation of binary classification tasks.

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  libraries ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ contain no packages

R[write to console]: Error in ROCR::prediction(predictor, response) : 
  Number of classes is not equal to 2.
ROCR currently supports only evaluation of binary classification tasks.

R[write 

2023-09-22 22:17:45,141 - Pipeline - INFO - Building LR model with all the features
2023-09-22 22:17:45,142 - Pipeline - INFO - Performing Hyperparameter optimisation
2023-09-22 22:17:48,256 - Pipeline - INFO - Building the model
2023-09-22 22:17:48,258 - Pipeline - INFO - Performing cross-validation
2023-09-22 22:17:49,379 - Pipeline - INFO - Building LR model with the selected features
2023-09-22 22:17:49,381 - Pipeline - INFO - Performing Hyperparameter optimisation
2023-09-22 22:17:50,155 - Pipeline - INFO - Building the model
2023-09-22 22:17:50,156 - Pipeline - INFO - Performing cross-validation
2023-09-22 22:17:51,209 - Pipeline - INFO - Building LGBM model with all the features
2023-09-22 22:17:51,211 - Pipeline - INFO - Performing Hyperparameter optimisation
2023-09-22 22:18:10,096 - Pipeline - INFO - Building the model
2023-09-22 22:18:10,101 - Pipeline - INFO - Performing cross-validation
2023-09-22 22:18:13,198 - Pipeline - INFO - Building LGBM model with the selected featu

...

2023-09-22 23:05:50,519 - Pipeline - INFO - Building Ensemble LR model with all the features
2023-09-22 23:05:50,519 - Pipeline - INFO - Performing Hyperparameter optimisation
2023-09-22 23:05:51,069 - Pipeline - INFO - Building the model
2023-09-22 23:05:51,071 - Pipeline - INFO - Performing cross-validation
2023-09-22 23:05:51,943 - Pipeline - INFO - Saving the CV results for all the models
2023-09-22 23:05:51,947 - Pipeline - INFO - Completed !!!
2023-09-22 23:05:52,150 - Pipeline - INFO - dirName: 95_5_samples_icd_vb_0_va_72_lb_0_la_72
2023-09-22 23:05:52,151 - Pipeline - INFO - Running Predictions for vb_0_va_72_lb_0_la_72, targetStart : 0, targetEnd : 7
2023-09-22 23:05:52,151 - Pipeline - INFO - Reading data
2023-09-22 23:05:52,411 - Pipeline - INFO - Formatting data
2023-09-22 23:05:52,419 - Pipeline - INFO - Performing SFS
2023-09-22 23:07:17,134 - Pipeline - INFO - Building XGBoost model with all the features
2023-09-22 23:07:17,136 - Pipeline - INFO - Performing Hyperparamet

R[write to console]: Error in ROCR::prediction(predictor, response) : 
  Number of classes is not equal to 2.
ROCR currently supports only evaluation of binary classification tasks.

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  libraries ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ contain no packages

R[write to console]: 2: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  libraries ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ contain no packages

R[write to console]: 3: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  libraries ‘/usr/local/lib/R/site-library’, 

2023-09-22 23:09:35,547 - Pipeline - INFO - Building XGBoost model with the selected features
2023-09-22 23:09:35,548 - Pipeline - INFO - Performing Hyperparameter optimisation
2023-09-22 23:09:35,548 - Pipeline - INFO - Building the model
2023-09-22 23:09:35,549 - Pipeline - INFO - Hyperparameter optimisation for: {'max_depth': range(1, 10), 'scale_pos_weight': [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]}
2023-09-22 23:09:59,737 - Pipeline - INFO - Hyperparameter optimisation for: {'n_estimators': range(50, 250, 10)}
2023-09-22 23:10:11,704 - Pipeline - INFO - Hyperparameter optimisation for: {'min_child_weight': range(1, 10)}
2023-09-22 23:10:15,206 - Pipeline - INFO - Hyperparameter optimisation for: {'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]}
2023-09-22 23:10:17,937 - Pipeline - INFO - Hyperparameter optimisation for: {'subsample': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}
2023-09-22 23:10:51,444 - Pipeline - INFO - Hyperp

R[write to console]: Error in ROCR::prediction(predictor, response) : 
  Number of classes is not equal to 2.
ROCR currently supports only evaluation of binary classification tasks.

R[write to console]: In addition: 
R[write to console]: There were 36 warnings (use warnings() to see them)
R[write to console]: 

R[write to console]: Error in ROCR::prediction(predictor, response) : 
  Number of classes is not equal to 2.
ROCR currently supports only evaluation of binary classification tasks.

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  libraries ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ contain no packages

R[write to console]: Error in ROCR::prediction(predictor, response) : 
  Number of classes is not equal to 2.
ROCR currently supports only evaluation of binary classification tasks.

R[write 

2023-09-22 23:14:54,307 - Pipeline - INFO - Building Ensemble LR model with all the features
2023-09-22 23:14:54,308 - Pipeline - INFO - Performing Hyperparameter optimisation
2023-09-22 23:14:54,866 - Pipeline - INFO - Building the model
2023-09-22 23:14:54,867 - Pipeline - INFO - Performing cross-validation
2023-09-22 23:14:55,801 - Pipeline - INFO - Saving the CV results for all the models
2023-09-22 23:14:55,803 - Pipeline - INFO - Completed !!!


## Remove the data matrix file from working directory

In [8]:
import os


os.system(
    '''rm ''' + dataDirName + '''data_matrix.csv'''
)

0